In [1]:
import numpy as np
from astropy.table import Table, join, setdiff

In [2]:
gedr3_agn = Table.read("/Users/Neo/Astronomy/data/catalogs/gaia/edr3/gedr3_agn.fits")

mask1 = (gedr3_agn['catalogue_name'] == 'ICRF3 S/X                                 ')
mask2 = (gedr3_agn['catalogue_name'] == 'ICRF3 K                                   ')
mask3 = (gedr3_agn['catalogue_name'] == 'ICRF3 X/Ka                                ')
mask = (mask1 | mask2 | mask3)
gedr3_source = gedr3_agn[mask]

print("There are {} ICRF3 sources in EDR3:".format(len(gedr3_source)))
print("SX:  {}".format(len(np.nonzero(mask1)[0])))
print("K:   {}".format(len(np.nonzero(mask2)[0])))
print("XKa: {}".format(len(np.nonzero(mask3)[0])))

There are 3181 ICRF3 sources in EDR3:
SX:  3142
K:   24
XKa: 15


In [3]:
gcrf3_sou = Table.read('/Users/Neo/Astronomy/data/catalogs/gaia/edr3/gedr3_frame_rotator_source.fits')

gcrf3_source = gcrf3_sou[gcrf3_sou['considered_for_reference_frame_orientation']]
gcrf3_source = gcrf3_source['source_id', 
                           'considered_for_reference_frame_orientation',
                           'used_for_reference_frame_orientation',
                           'considered_for_reference_frame_spin',
                           'used_for_reference_frame_spin']

In [4]:
gcrf3_icrf3_source = join(gedr3_source, gcrf3_source, keys='source_id', join_type='left')

In [5]:
tmp = gcrf3_icrf3_source['considered_for_reference_frame_orientation'].filled(False)
gcrf3_icrf3_source['considered_for_reference_frame_orientation'] = tmp

tmp = gcrf3_icrf3_source['used_for_reference_frame_orientation'].filled(False)
gcrf3_icrf3_source['used_for_reference_frame_orientation'] = tmp

tmp = gcrf3_icrf3_source['considered_for_reference_frame_spin'].filled(False)
gcrf3_icrf3_source['considered_for_reference_frame_spin'] = tmp

tmp = gcrf3_icrf3_source['used_for_reference_frame_spin'].filled(False)
gcrf3_icrf3_source['used_for_reference_frame_spin'] = tmp

In [6]:
# Find their ICRF name
from my_progs.catalog.read_icrf import read_icrf3
from astropy.table import Column

icrf3sx = read_icrf3(wv="sx")
sx_sou_name_i = icrf3sx['icrf_name', 'iers_name']
sx_sou_name_i.add_column(sx_sou_name_i['icrf_name'],
                         name="crossmatch_name")

icrf3k = read_icrf3(wv="k")
k_sou_name_i = icrf3k['icrf_name', 'iers_name']
k_sou_name_i.add_column(k_sou_name_i['icrf_name'],
                         name="crossmatch_name")

icrf3xka = read_icrf3(wv="xka")
xka_sou_name_i = icrf3xka['icrf_name', 'iers_name']
xka_sou_name_i.add_column(xka_sou_name_i['icrf_name'],
                         name="crossmatch_name")

sou_name = join(sx_sou_name_i, k_sou_name_i, join_type='outer')
sou_name = join(sou_name, xka_sou_name_i, join_type='outer')

In [7]:
sou_name_g = [sou_namei.strip() for sou_namei in gcrf3_icrf3_source['source_name_in_catalogue']]

gcrf3_icrf3_source.add_column(Column(sou_name_g),
                              name='crossmatch_name')

In [8]:
gcrf3_icrf3_source = join(gcrf3_icrf3_source, sou_name, 
                          keys='crossmatch_name')
gcrf3_icrf3_source.remove_column('crossmatch_name')

In [9]:
mask = (gcrf3_icrf3_source['astrometric_params_solved'] == 31)

len(gcrf3_icrf3_source[mask])

2301

In [10]:
mask = (gcrf3_icrf3_source['astrometric_params_solved'] == 95)

len(gcrf3_icrf3_source[mask])

880

In [11]:
mask = (gcrf3_icrf3_source['astrometric_params_solved'] == 2)

len(gcrf3_icrf3_source[mask])

0

In [12]:
mask = gcrf3_icrf3_source['used_for_reference_frame_orientation']

len(gcrf3_icrf3_source[mask])

2007

In [13]:
mask = gcrf3_icrf3_source['considered_for_reference_frame_orientation']

len(gcrf3_icrf3_source[mask])

2269

In [14]:
gcrf3_icrf3_source.write('/Users/Neo/Astronomy/data/catalogs/gaia/edr3/gaia_edr3_icrf3_source.fits',
                         overwrite=True)

In [15]:
mask1 = (gcrf3_icrf3_source['catalogue_name'] == 'ICRF3 S/X                                 ')
mask2 = (gcrf3_icrf3_source['catalogue_name'] == 'ICRF3 K                                   ')
mask3 = (gcrf3_icrf3_source['catalogue_name'] == 'ICRF3 X/Ka                                ')

print("There are {} ICRF3 sources in EDR3:".format(len(gcrf3_icrf3_source)))
print("SX:  {}".format(len(np.nonzero(mask1)[0])))
print("K:   {}".format(len(np.nonzero(mask2)[0])))
print("XKa: {}".format(len(np.nonzero(mask3)[0])))

There are 3181 ICRF3 sources in EDR3:
SX:  3142
K:   24
XKa: 15
